In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
import config

load_dotenv()

model = ChatOpenAI(model=config.model_name)     # 평가시 사용할 llm 모델은 성능 좋은 것을 써야 좀 더 정확한 평가가 가능
embedding_model = OpenAIEmbeddings(
    model=config.embedding_name
)


In [10]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

DOC_PATH = 'data/cleaned_all_restaurants.csv'

loader = CSVLoader(DOC_PATH)
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name=config.model_name, chunk_size=config.chunk_size, chunk_overlap=config.chunk_overlap
)
docs = loader.load_and_split(splitter)

vector_store=Chroma(
    embedding_function=embedding_model,
    collection_name=config.collection_name,
    persist_directory=config.persist_directory
)

In [11]:
vector_store._collection.count()

39104

In [25]:
docs

[Document(metadata={'source': 'data/cleaned_all_restaurants.csv', 'row': 0}, page_content='foodDetailTypes: 일반중식\nheaderInfo_nameKR: 홍보각\nheaderInfo_nameEN: \nheaderInfo_nameCN: 紅寶閣\nheaderInfo_bookYear: 2025\nheaderInfo_ribbonType: 3.0\ndefaultInfo_chefName: 여경래\ndefaultInfo_phone: 02-531-6479\ndefaultInfo_openHours: \ndefaultInfo_closeHours: \ndefaultInfo_openHoursWeekend: \ndefaultInfo_closeHoursWeekend: \ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: True\nstatusInfo_parking: 가능\nstatusInfo_creditCard: y\nstatusInfo_visit: \nstatusInfo_menu: 스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)\nstatusInfo_priceRange: 10~15만원대\nstatusInfo_openDate: 2007년\nstatusInfo_businessHours: 12:00~15:00/18:00~21:30(마지막 주문 20:00)\njuso_detailAddress: 노보텔앰배서더강남서울 LL층\njuso_roadAddrPart1: 서울특별시 강남구 봉은사로 130\njuso_engAddr: 130 Bongeunsa-ro, Gangnam-gu, Seoul\njuso_bdNm: 노보텔 앰배서더 강남 서울\njuso_siNm: 서울특별시\njuso_sggN

In [26]:
from pprint import pprint
pprint(docs[0].page_content)

('foodDetailTypes: 일반중식\n'
 'headerInfo_nameKR: 홍보각\n'
 'headerInfo_nameEN: \n'
 'headerInfo_nameCN: 紅寶閣\n'
 'headerInfo_bookYear: 2025\n'
 'headerInfo_ribbonType: 3.0\n'
 'defaultInfo_chefName: 여경래\n'
 'defaultInfo_phone: 02-531-6479\n'
 'defaultInfo_openHours: \n'
 'defaultInfo_closeHours: \n'
 'defaultInfo_openHoursWeekend: \n'
 'defaultInfo_closeHoursWeekend: \n'
 'defaultInfo_dayOff: 연중무휴\n'
 'defaultInfo_app2Yn: True\n'
 'statusInfo_parking: 가능\n'
 'statusInfo_creditCard: y\n'
 'statusInfo_visit: \n'
 'statusInfo_menu: 스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 '
 '6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), '
 '셰프스페셜송이특찜(15만원)\n'
 'statusInfo_priceRange: 10~15만원대\n'
 'statusInfo_openDate: 2007년\n'
 'statusInfo_businessHours: 12:00~15:00/18:00~21:30(마지막 주문 20:00)\n'
 'juso_detailAddress: 노보텔앰배서더강남서울 LL층\n'
 'juso_roadAddrPart1: 서울특별시 강남구 봉은사로 130\n'
 'juso_engAddr: 130 Bongeunsa-ro, Gangnam-gu, Seoul\n'
 'juso_bdNm: 노보텔 앰배서더 강남 서울\n'
 'juso_siN

## 평가

In [13]:
import random
total_sampes = 5    # 추출할 샘플 개수
eval_context_list = []  # Sample들을 담을 리스트
while len(eval_context_list) < 5:
    _context = docs[random.randint(0, len(docs)-1)].page_content
    if len(_context) < 100: # 글자수가 너무 적으면 질문을 생성 못하기 때문에 글자수 체크
        continue
    eval_context_list.append(_context)


In [28]:
# Chain을 이용해서 LLM에게 질문-답 생성을 요청
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from textwrap import dedent

# JSONOutputParser에서 사용할 스키마 생성
class EvaluationDatasetSchema(BaseModel):
    user_input:str=Field(...,description="질문(Question)")
    retrieved_contexts:list[str] = Field(...,description="LLM이 답변할 때 참조할 context")
    reference:str = Field(...,description="정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvaluationDatasetSchema)
# print(parser.get_format_instructions())

prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개의 질문을 생성하세요. 
        
        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [context]에 없는 내용을 가지고 질문-답변을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 답변을 만들고 그 내용이 [context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어주십시오.
                                
        출력 형식: {format_instructions}
                    
        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model='gpt-4o')
dataset_generator_chain = prompt_template | model | parser 

In [29]:
c = eval_context_list[0]
qa = dataset_generator_chain.invoke({"context":c,"num_questions":5})

In [30]:
qa

[{'user_input': '호호닭발의 영업 시간은?',
  'retrieved_contexts': ['defaultInfo_openHours: 15:00',
   'defaultInfo_closeHours: 03:00',
   'statusInfo_businessHours: 15:00~03:00(익일)'],
  'reference': '호호닭발의 영업 시간은 오후 3시부터 다음 날 오전 3시까지입니다.'},
 {'user_input': '호호닭발의 가격대는?',
  'retrieved_contexts': ['statusInfo_priceRange: 1~2만원대'],
  'reference': '호호닭발의 가격대는 1~2만원대입니다.'},
 {'user_input': '호호닭발에서 주차가 가능한가요?',
  'retrieved_contexts': ['statusInfo_parking: 불가'],
  'reference': '호호닭발에서는 주차가 불가능합니다.'},
 {'user_input': '호호닭발에서 인기 있는 메뉴는?',
  'retrieved_contexts': ['statusInfo_menu: 뼈있는닭발(1만8천원), 뼈없는닭발(1만9천원), 국물닭발(뼈있는닭발 2만원, 뼈없는닭발 2만1천원), 오돌뼈(1만5천원)'],
  'reference': '호호닭발에서는 뼈있는 닭발, 뼈없는 닭발, 국물 닭발 등이 인기 메뉴입니다.'},
 {'user_input': '호호닭발에 어떻게 가나요?',
  'retrieved_contexts': ['statusInfo_visit: 건대입구역 1번 출구 앞 골목으로 270m 직진'],
  'reference': '호호닭발은 건대입구역 1번 출구 앞 골목으로 270m 직진하면 됩니다.'}]

In [31]:
# 확인 후 retrieved_contexts를 context로 변경
for d in qa:
    d['retrieved_contexts'] = c
qa

[{'user_input': '호호닭발의 영업 시간은?',
  'retrieved_contexts': 'foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\nheaderInfo_nameEN: \nheaderInfo_nameCN: \nheaderInfo_bookYear: \nheaderInfo_ribbonType: \ndefaultInfo_chefName: \ndefaultInfo_phone: 02-465-3455\ndefaultInfo_openHours: 15:00\ndefaultInfo_closeHours: 03:00\ndefaultInfo_openHoursWeekend: \ndefaultInfo_closeHoursWeekend: \ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: False\nstatusInfo_parking: 불가\nstatusInfo_creditCard: \nstatusInfo_visit: 건대입구역 1번 출구 앞 골목으로 270m 직진\nstatusInfo_menu: 뼈있는닭발(1만8천원), 뼈없는닭발(1만9천원), 국물닭발(뼈있는닭발 2만원, 뼈없는닭발 2만1천원), 오돌뼈(1만5천원)\nstatusInfo_priceRange: 1~2만원대\nstatusInfo_openDate: \nstatusInfo_businessHours: 15:00~03:00(익일)\njuso_detailAddress: \njuso_roadAddrPart1: 서울특별시 광진구 동일로24길 96\njuso_engAddr: 96, Dongil-ro 24-gil, Gwangjin-gu, Seoul\njuso_bdNm: \njuso_siNm: 서울특별시\njuso_sggNm: 광진구\njuso_emdNm: 화양동\njuso_liNm: \njuso_rn: 동일로24길\njuso_buldMnnm: 96\njuso_buldSlno: 0\nreview_review: 닭발로 유명한 곳. 닭발은 매콤하며 쫄깃쫄깃하다

In [32]:
import pandas as pd
pd.DataFrame(qa)

,user_input,retrieved_contexts,reference
0,호호닭발의 영업 시간은?,foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\n...,호호닭발의 영업 시간은 오후 3시부터 다음 날 오전 3시까지입니다.
1,호호닭발의 가격대는?,foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\n...,호호닭발의 가격대는 1~2만원대입니다.
2,호호닭발에서 주차가 가능한가요?,foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\n...,호호닭발에서는 주차가 불가능합니다.
3,호호닭발에서 인기 있는 메뉴는?,foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\n...,"호호닭발에서는 뼈있는 닭발, 뼈없는 닭발, 국물 닭발 등이 인기 메뉴입니다."
4,호호닭발에 어떻게 가나요?,foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\n...,호호닭발은 건대입구역 1번 출구 앞 골목으로 270m 직진하면 됩니다.


In [33]:
### 전체 context sample들로 qa dataset을 생성
eval_data_list = []
num_questions = 5   # context당 k(5)개 QA 쌍 생성
for context in eval_context_list:
    _eval_data_list=dataset_generator_chain.invoke({"context":context,"num_questions":num_questions})
    # retrieve_contexts의 값을 변경
    for eval_data in _eval_data_list:
        eval_data['retrieved_contexts']=[context]
    eval_data_list.extend(_eval_data_list)

In [35]:
len(eval_data_list)
eval_data_list[:2]

[{'user_input': '호호닭발의 영업 시간은 언제인가요?',
  'retrieved_contexts': ['foodDetailTypes: 닭발\nheaderInfo_nameKR: 호호닭발\nheaderInfo_nameEN: \nheaderInfo_nameCN: \nheaderInfo_bookYear: \nheaderInfo_ribbonType: \ndefaultInfo_chefName: \ndefaultInfo_phone: 02-465-3455\ndefaultInfo_openHours: 15:00\ndefaultInfo_closeHours: 03:00\ndefaultInfo_openHoursWeekend: \ndefaultInfo_closeHoursWeekend: \ndefaultInfo_dayOff: 연중무휴\ndefaultInfo_app2Yn: False\nstatusInfo_parking: 불가\nstatusInfo_creditCard: \nstatusInfo_visit: 건대입구역 1번 출구 앞 골목으로 270m 직진\nstatusInfo_menu: 뼈있는닭발(1만8천원), 뼈없는닭발(1만9천원), 국물닭발(뼈있는닭발 2만원, 뼈없는닭발 2만1천원), 오돌뼈(1만5천원)\nstatusInfo_priceRange: 1~2만원대\nstatusInfo_openDate: \nstatusInfo_businessHours: 15:00~03:00(익일)\njuso_detailAddress: \njuso_roadAddrPart1: 서울특별시 광진구 동일로24길 96\njuso_engAddr: 96, Dongil-ro 24-gil, Gwangjin-gu, Seoul\njuso_bdNm: \njuso_siNm: 서울특별시\njuso_sggNm: 광진구\njuso_emdNm: 화양동\njuso_liNm: \njuso_rn: 동일로24길\njuso_buldMnnm: 96\njuso_buldSlno: 0\nreview_review: 닭발로 유명한 곳. 닭발은 매콤하며

In [36]:
eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

(25, 3)

In [38]:
eval_df.head()
eval_df.tail()

,user_input,retrieved_contexts,reference
20,삼송빵집의 영업시간은 언제인가요?,[foodDetailTypes: 베이커리\nheaderInfo_nameKR: 삼송빵...,삼송빵집의 영업시간은 오전 8시부터 오후 10시까지입니다. 재료가 소진되면 영업을 ...
21,삼송빵집의 인기 메뉴는 무엇인가요?,[foodDetailTypes: 베이커리\nheaderInfo_nameKR: 삼송빵...,삼송빵집의 인기 메뉴는 통옥수수빵과 구운고로케입니다.
22,삼송빵집의 위치는 어디인가요?,[foodDetailTypes: 베이커리\nheaderInfo_nameKR: 삼송빵...,삼송빵집은 대구광역시 중구 중앙대로 397에 위치해 있습니다.
23,삼송빵집의 주차 가능 여부는?,[foodDetailTypes: 베이커리\nheaderInfo_nameKR: 삼송빵...,삼송빵집은 주차가 불가능합니다.
24,삼송빵집의 명절 휴무일은 언제인가요?,[foodDetailTypes: 베이커리\nheaderInfo_nameKR: 삼송빵...,삼송빵집은 명절 당일에 휴무입니다.


In [39]:
eval_df['user_input']

0         호호닭발의 영업 시간은 언제인가요?
1            호호닭발의 위치는 어디인가요?
2         호호닭발의 대표 메뉴는 무엇인가요?
3           호호닭발에서 주차가 가능한가요?
4          호호닭발의 전화번호는 무엇인가요?
5      모멘토스는 어떤 음식을 전문으로 하나요?
6          모멘토스의 전화번호는 무엇인가요?
7           모멘토스에서 주차가 가능한가요?
8         모멘토스의 추천 메뉴는 무엇인가요?
9            모멘토스의 주소는 어디인가요?
10       신림춘천집의 인기 메뉴는 무엇인가요?
11          신림춘천집의 주차 가능 여부는?
12     신림춘천집의 운영 시간은 어떻게 되나요?
13          신림춘천집의 주소는 어디인가요?
14         신림춘천집의 휴무일은 언제인가요?
15         진사향의 셰프 이름은 무엇인가요?
16    진사향의 주말 영업 시간은 어떻게 되나요?
17       진사향의 대표적인 메뉴는 무엇인가요?
18       진사향은 어느 호텔에 위치해 있나요?
19       진사향의 시그니처 메뉴는 무엇인가요?
20         삼송빵집의 영업시간은 언제인가요?
21        삼송빵집의 인기 메뉴는 무엇인가요?
22           삼송빵집의 위치는 어디인가요?
23           삼송빵집의 주차 가능 여부는?
24       삼송빵집의 명절 휴무일은 언제인가요?
Name: user_input, dtype: object

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

# 평가 알로리즘 모듈
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field


# 메모리 관련 모듈
from langchain_core.chat_history import InMemoryChatMessageHistory

from langchain.chains import RetrievalQA
from langchain.schema import AIMessage, HumanMessage




from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

In [ ]:
print(len(context_list), len(response_list))
# pprint(context_list[:2])
# pprint(response_list[:2])

In [ ]:
response_list

In [ ]:
eval_df["retrieved_contexts"] = context_list # context 추가
eval_df["response"] = response_list   # 정답 추가

In [ ]:
# Dataframe으로 부터 EvalDataset 생성
eval_dataset = EvaluationDataset.from_pandas(eval_df)
eval_dataset

In [ ]:
# HuggingFace에 업로드 -> dataets.Dataset으로 변환
eval_dataset.to_hf_dataset()#.push_to_hub()

In [ ]:
# model_name = "gpt-4o"
model_name = "gpt-4o-mini"
model = ChatOpenAI(model=model_name)
eval_llm = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)


## GPT-4o-mini 모델을 사용하여 평가 
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)

In [ ]:
result

In [ ]:
# gpt-4o : 분당 토큰 리미트에 걸려 RateLimitError 가 발생할 수있다. gpt-40: 30,000 TPM, gpt-4o-mini: 200,000 TPM
# 또한 network 연결등 문제가 발생하면 timeout 이 되어 평가가 실패할 수 있다.
# LLM 연결과 관련해 timetout이나 ratelimiterror 발생시 metrics를 나눠서 실행, 설정 변경을 통해 해결한다.
## https://platform.openai.com/settings/organization/limits
run_config = RunConfig(
    timeout=360,     # LLM 호출 이후 최대 대기 시간. 지정한 초까지 응답을 기다린다. 
    max_retries=20, # API 호출시 지정한 횟수만큼 재시도 한다.
    max_wait=360,   # 재시도 대기 시간(초) 180초 기다린 후 재시도 한다.
    max_workers=1   # 병렬처리 worker 수. 1로 설정하면 순차적으로 처리한다. (default: 16)
)
metrics1 = [
    LLMContextRecall(llm=eval_llm),
]
metrics2 = [
    LLMContextPrecisionWithReference(llm=eval_llm),
]
metrics3 = [
    Faithfulness(llm=eval_llm),
]
metrics4 = [
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]

In [ ]:
result1 = evaluate(dataset=eval_dataset, metrics=metrics1, run_config=run_config)
result1

In [ ]:
result2 = evaluate(dataset=eval_dataset, metrics=metrics2, run_config=run_config)
result2

In [ ]:
result3 = evaluate(dataset=eval_dataset, metrics=metrics3, run_config=run_config)
result3

In [ ]:
result4 = evaluate(dataset=eval_dataset, metrics=metrics4, run_config=run_config)
result4